In [1]:
# -*- coding: utf-8 -*-

"""
Author: Rien Sonck
Date: 2020-06-28
Description: This file aims to recover the parameters used to generate simulated data.  
"""
######################
# Importing packages #
######################
from Sync_Model import model_sim
from scipy.optimize import differential_evolution
import pandas as pd
from scipy import ndimage
import numpy as np
import time


In [2]:
def new_goal_func(x, *args):
    ######################################
    # Value Extraction and Initalization #
    ######################################
    """ 
    function that returns the error of the goal function based on previously generated 
    simulated data files by Sync_Model_Simulations.py

    Parameters
    ----------
    x    : integer
           this value lies in-between the specified bounds, and will be generated by the differential evolution algorithm. 
    args : tuple
           any additional fixed model parameters and other variables needed to completely specify the objective function.
    """ 
    gd = args[0]              # gamma distance model parameter
    squared_err = []
    mfc, thres = x[0], x[1]
    ####################
    # Reading in data #
    ###################
    # reading in the target data
    pd_targetData = pd.DataFrame(args[1])
    ################################
    # Generating model simulations #
    ################################
    pd_data = model_sim(Threshold = thres, drift = gd, theta_freq = mfc, save_behavioral = False, return_dataframe = True)
    pd_targetData = pd_targetData.rename(columns={'rts': 'rt', 'corr_resp': 'accuracy'})
    pd_targetData['rt'] = pd_targetData['rt'] * 1000 # turning seconds into miliseconds
    pd_targetData = pd_targetData.loc[pd_targetData['instructs'] == 0] 
    ###################################
    # Calculating RT Shape difference #
    ###################################
    accuracy = pd_data['accuracy'].unique() 
    for acc in accuracy: 
        # extracting RTs
        rt_sim = pd_data.loc[pd_data['accuracy'] == acc, 'rt'].values                     # simulated RTs
        rt_target = pd_targetData.loc[pd_targetData['accuracy'] == acc, 'rt'].values    # target RTs
        # creating bins
        bins = np.arange(0, 700.1, 25)
        # getting histogram data
        heights_sim, bins = np.histogram(rt_sim, bins=bins)
        heights_target, bins = np.histogram(rt_target, bins=bins)
        # calculating the difference
        diff = heights_target - heights_sim
        # calculating the squared error
        err = sum(diff**2)
        squared_err.append(err)

    ###################################
    # Calculating RT Peak difference  #
    ###################################
    # applying kernels to the error and correct RT distributions
    # getting the right sigma for smoothing
    cor_indices_target, err_indices_target = [], []
    cor_indices_sim, err_indices_sim = [], []
    bins = np.arange(0, 1000, 10)

    sigma_cor = 5
    sigma_err = 5
    # this while loop will keep reducing the sigma until we can recover the second peak
    while len(cor_indices_target) < 1 or len(err_indices_target) < 1:
        RT_hist_list_target = RT_distribution_smoothing(pd_targetData, bins, sigma_err, sigma_cor)
        # finding where in the bins the peaks and troughs happen and mark it 
        time_step_change_target = np.diff(np.sign(np.diff(RT_hist_list_target)))
        # extracting the index of the peak and trough bins based on the marker
        cor_indices_target = np.where((time_step_change_target[1]!= 0) & (time_step_change_target[1] != 1))[0]
        err_indices_target = np.where((time_step_change_target[0] != 0) & (time_step_change_target[0] != 1))[0] 
        if len(cor_indices_target) < 1: 
            sigma_cor = sigma_cor - 1
        if len(err_indices_target) < 1:
            sigma_err = sigma_err - 1


    sigma_cor = 5
    sigma_err = 5
    while len(cor_indices_sim) < 1 or len(err_indices_sim) < 1:
        RT_hist_list_sim = RT_distribution_smoothing(pd_data, bins, sigma_err, sigma_cor)
        # finding where in the bins the peaks and troughs happen and mark it 
        time_step_change_sim = np.diff(np.sign(np.diff(RT_hist_list_sim)))
        # extracting the index of the peak and trough bins based on the marker
        cor_indices_sim = np.where((time_step_change_sim[1]!= 0) & (time_step_change_sim[1] != 1))[0]
        err_indices_sim = np.where((time_step_change_sim[0] != 0) & (time_step_change_sim[0] != 1))[0]
        if len(cor_indices_sim) < 1: 
            sigma_cor = sigma_cor - 1
        if len(err_indices_sim) < 1:
            sigma_err = sigma_err - 1

    # extracting the peaks and trough bin value
    peaks_troughs_err_target = RT_hist_list_target[0][err_indices_target]
    peaks_troughs_cor_target = RT_hist_list_target[1][cor_indices_target]
    peaks_troughs_err_sim = RT_hist_list_sim[0][err_indices_sim]
    peaks_troughs_cor_sim = RT_hist_list_sim[1][cor_indices_sim]
    # calculate the relative peak height
    #relative_height_cor_target = peaks_troughs_cor_target[1] / peaks_troughs_cor_target[0]
    #relative_height_err_target = peaks_troughs_err_target[1] / peaks_troughs_err_target[0]
    #relative_height_cor_sim = peaks_troughs_cor_sim[1] / peaks_troughs_cor_sim[0] 
    #relative_height_err_sim = peaks_troughs_err_sim[1] / peaks_troughs_err_sim[0]
    relative_height_target = peaks_troughs_err_target[0] / peaks_troughs_cor_target[0]
    relative_height_sim = peaks_troughs_err_sim[0] / peaks_troughs_cor_sim[0]
    # error
    #peak_error = np.abs(relative_height_cor_sim - relative_height_cor_target) + np.abs(relative_height_err_sim - relative_height_err_target)
    peak_error = np.abs(relative_height_target - relative_height_sim)


    # squared_err[1] is error, squared_err[0] is correct
    # peak_error * 10**6 -
    error = squared_err[1] + squared_err[0] + (peak_error * 10**4)
    print("mfc: {0}, thres: {1}, gd: {2}, error: {3}".format(mfc, thres, gd, error))
    return error

In [3]:
def RT_distribution_smoothing(pd_data, bins, sigma_err, sigma_cor):
    """ 
    Function that applies the difference kernel and then a Gaussian smoothing kernel to the error 
    and correct RT distributions corresponding to each MFC frequency in the model.  

    Parameters
    ----------
    pd_data    : pandas dataframe 
                 dataframe containing the data
    bins       : numpy array
                 array containing the bins for the histograms
    sigma      : integer
                 smoothing of the Gaussian kernel
    """ 
    accuracy = [0, 1]
    thetas_hist_list = []
    theta_hist = np.zeros(shape=[2, 1, len(bins)-1])
    ###################
    # Reading in data #
    ###################
    # creating histograms
    theta_hist[0, :] = np.histogram(pd_data.loc[pd_data['accuracy']== 0, 'rt'].values, bins=bins, normed = True)[0] # error histograms
    theta_hist[1, :] = np.histogram(pd_data.loc[pd_data['accuracy']== 1, 'rt'].values, bins=bins, normed = True)[0] # correct histograms
    ###################################
    # Difference Kernel and Smoothing #
    ###################################
    # run difference kernel
    theta_histDiff_err = -np.array([theta_hist[0][0][bin_n] - (theta_hist[0][0][bin_n-1] + theta_hist[0][0][bin_n+1]).mean() for bin_n in np.arange(1, len(bins)-2)]).T
    theta_histDiff_cor = -np.array([theta_hist[1][0][bin_n] - (theta_hist[1][0][bin_n-1] + theta_hist[1][0][bin_n+1]).mean() for bin_n in np.arange(1, len(bins)-2)]).T
    # smooth
    theta_histDiffSmooth_err = ndimage.gaussian_filter1d(theta_histDiff_err, axis = 0, sigma = sigma_err)
    theta_histDiffSmooth_cor = ndimage.gaussian_filter1d(theta_histDiff_cor, axis = 0, sigma = sigma_cor)
    return (theta_histDiffSmooth_err, theta_histDiffSmooth_cor)

In [ ]:
########################
# Value initialization #
########################
path = ''
file = '{0}behav_data_34obs.npy'.format(path)
# data is a list of dictionnaries, each dictionnary is one participant
data = np.load(file, allow_pickle=True)
gd = 10 # fixing the gamma distance 
subject = 2
args = (gd, data[subject]) # arguments that the sync_func function needs to run (gamma distance, MFC frequency value, target dataframe)
# model parameters
bounds = [(3, 9), (1,5)]

#################
# Model Fitting #
#################
t = time.time()
result = differential_evolution(func=new_goal_func, bounds=bounds, args=args, workers = -1, popsize = 15, maxiter = 1000, updating = 'deferred', tol = 2, atol = 0)
print("mfc_result = {2}, thres_result = {3}\n".format(result.x[0], result.x[1]))
print('\ttime taken: %.2fmin' % ((time.time() - t) / 60.))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


mfc: 5.052243862479101, thres: 2.3518177895109966, gd: 10, error: 59681.11545779649


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


mfc: 5.8572953586403385, thres: 4.423478259915116, gd: 10, error: 46123.09409668993
mfc: 8.876108136364829, thres: 1.7399681211452647, gd: 10, error: 75573.01517967055
mfc: 8.548759841457011, thres: 3.2247498883613783, gd: 10, error: 57972.42124631202
mfc: 4.4945850125179465, thres: 4.009992504492818, gd: 10, error: 48796.49172466091
mfc: 3.4389782714758033, thres: 1.3374807870824783, gd: 10, error: 122249.24641578541
mfc: 4.3149991066368685, thres: 2.290392887920105, gd: 10, error: 70596.6530030673
mfc: 7.61119701665189, thres: 3.70883650973431, gd: 10, error: 50338.30338947695
mfc: 3.0891236029125784, thres: 2.489017730226954, gd: 10, error: 58096.106772984764
mfc: 6.903370030786662, thres: 3.8330197397925057, gd: 10, error: 48798.82342094565
mfc: 5.388074809963578, thres: 1.8491851700315056, gd: 10, error: 80030.3970802863
mfc: 8.773905001662978, thres: 3.5324345243821815, gd: 10, error: 50850.04914030352
mfc: 8.033100954951205, thres: 3.59058595401535, gd: 10, error: 46994.52705768